In [1]:
import sys
sys.path.append("../")

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import os
import matplotlib.pyplot as plt

from datasets.llff import LLFFDataset
from datasets.llff import *

In [4]:
root_dir = "/scratch/saksham/data/freiburg_small/"

In [5]:
!ls $data_path

 01_llfm_dataloader_breakdown-Copy1.ipynb   ndc.ipynb
 01_llfm_dataloader_breakdown.ipynb	    plot2D.mp4
 01_llfm_tum_pose_t.ipynb		    plot2D_corrected.mp4
 01_tum_dataloader.ipynb		    plot3D.mp4
 02_test_tum_dataset.ipynb		    plot3DCamera.mp4
 03_evaluate_model_fern.ipynb		    plot3DCamera_corrected.mp4
 04_test_tum.ipynb			    plot3D_corrected.mp4
'[EXP] nerf_walkthrough.ipynb'		    test.ipynb
 colmap.npy				    tum.npy
 colmap_vs_tum.ipynb			    tum_aligned.npy


In [6]:
from datasets.colmap_utils import \
    read_cameras_binary, read_images_binary, read_points3d_binary

In [8]:
camdata = read_cameras_binary(os.path.join(root_dir, 'sparse/0/cameras.bin'))
img_wh = (80,  60)
H = camdata[1].height
W = camdata[1].width
focal = camdata[1].params[0] * img_wh[0]/W

In [9]:
imdata = read_images_binary(os.path.join(root_dir, 'sparse/0/images.bin'))

In [10]:
perm = np.argsort([imdata[k].name for k in imdata])

In [11]:
image_paths = [os.path.join(root_dir, 'images', name)
                            for name in sorted([imdata[k].name for k in imdata])]

In [13]:
w2c_mats = []
bottom = np.array([0, 0, 0, 1.]).reshape(1, 4)
for k in imdata:
    im = imdata[k]
    R = im.qvec2rotmat()
    t = im.tvec.reshape(3, 1)
    w2c_mats += [np.concatenate([np.concatenate([R, t], 1), bottom], 0)]
w2c_mats = np.stack(w2c_mats, 0)
poses = np.linalg.inv(w2c_mats)[:, :3] # (N_images, 3, 4) cam2world matrices

In [14]:
bounds = np.zeros((len(poses), 2)) # (N_images, 2)
pts3d = read_points3d_binary(os.path.join(root_dir, 'sparse/0/points3D.bin'))

In [15]:
pts_world = np.zeros((1, 3, len(pts3d))) # (1, 3, N_points)
visibilities = np.zeros((len(poses), len(pts3d))) # (N_images, N_points)
for i, k in enumerate(pts3d):
    pts_world[0, :, i] = pts3d[k].xyz
    for j in pts3d[k].image_ids:
        visibilities[j-1, i] = 1

In [16]:
depths = ((pts_world-poses[..., 3:4])*poses[..., 2:3]).sum(1) # (N_images, N_points)
for i in range(len(poses)):
    visibility_i = visibilities[i]
    zs = depths[i][visibility_i==1]
    bounds[i] = [np.percentile(zs, 0.1), np.percentile(zs, 99.9)]
# permute the matrices to increasing order
poses = poses[perm]
bounds = bounds[perm]

In [17]:
poses = np.concatenate([poses[..., 0:1], -poses[..., 1:3], poses[..., 3:4]], -1)

In [20]:
bounds.min(), bounds.max()

(2.0559904843130434, 28.39163776138985)

In [24]:
poses = poses[:30]
image_paths = image_paths[:30]
bounds = bounds[:30]

In [25]:
bounds.max()

28.39163776138985

In [18]:
poses, _ = center_poses(poses)

In [19]:
distances_from_center = np.linalg.norm(poses[..., 3], axis=1)

In [20]:
distances_from_center

array([0.95737697, 0.95691527, 0.99753154, 1.00934402, 0.99352704,
       0.97806374, 0.98133264, 0.940594  , 0.87972338, 0.79775828,
       0.69448124, 0.6285807 , 0.56177675, 0.47512866, 0.40115907,
       0.31781431, 0.2843643 , 0.29187976, 0.40513849, 0.4563209 ,
       0.60194595, 0.70305226, 0.7594199 , 0.88258841, 0.99679348,
       1.13806552, 1.29228361, 1.46907967, 1.638834  , 1.76030004])

In [21]:
val_idx = np.argmin(distances_from_center) # choose val image as the closest to
                                                   # center image

In [22]:
near_original = bounds.min()

In [23]:
near_original

2.0559904843130434

In [24]:
1./(near_original * 0.75)

0.6485114320841969

In [25]:
near_original*(1./(near_original * 0.75))

1.3333333333333335

In [26]:
scale_factor = near_original*0.75 # 0.75 is the default parameter
                                          # the nearest depth is at 1/0.75=1.33

In [35]:
near_original = bounds.min()
scale_factor = near_original*0.75 

In [37]:
bounds /= scale_factor
poses[..., 3] /= scale_factor

In [48]:
img_wh=(504, 378)

directions = get_ray_directions(img_wh[1], img_wh[0], focal) # (H, W, 3)